# 股票資料抓取與分析 📈
## 使用Data Science常用方法抓取1分鐘間隔的股票資訊

這個notebook展示如何使用最普遍的data science工具和方法來抓取股票的即時資料，包括：
- 使用yfinance API抓取股票資料
- 1分鐘間隔的高頻資料
- 技術指標計算
- 資料視覺化分析
- 多股票對比分析

**作者**: Data Scientist  
**最後更新**: 2025-07-04

## 1. 安裝與匯入必要套件 📦

首先安裝並匯入所有需要的Python套件。這些是data scientist最常用的工具：
- **yfinance**: 最受歡迎的股票資料API
- **pandas**: 資料處理和分析
- **numpy**: 數值計算
- **matplotlib & seaborn**: 資料視覺化
- **datetime**: 時間處理

In [ ]:
# 如果還沒安裝這些套件，請先執行以下命令
# !pip install yfinance pandas numpy matplotlib seaborn

print("正在安裝必要套件...")

In [ ]:
# 匯入所有必要的套件
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import json

# 設定顯示選項
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 設定中文字體支援
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定圖表樣式
plt.style.use('default')
sns.set_palette("husl")

# 忽略警告訊息
warnings.filterwarnings('ignore')

print("✅ 所有套件匯入成功！")
print(f"📅 執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python版本: {pd.__version__} (pandas)")
print(f"📊 yfinance版本: {yf.__version__}")

## 2. 設定股票代碼與時間區間 ⚙️

設定我們要分析的股票代碼和時間範圍。我們將包含：
- **美股**: Apple、Microsoft、Tesla等熱門股票
- **台股**: 台積電、鴻海等
- **加密貨幣**: Bitcoin、Ethereum
- **時間範圍**: 最近1天的1分鐘間隔資料

In [ ]:
# 定義要分析的股票代碼
stock_symbols = {
    '美股科技股': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA'],
    '美股其他': ['SPY', 'QQQ', 'IWM', 'GLD', 'SLV'],
    '台股': ['2330.TW', '2317.TW', '2454.TW', '1303.TW', '2308.TW'],
    '加密貨幣': ['BTC-USD', 'ETH-USD', 'BNB-USD']
}

# 合併所有股票代碼
all_symbols = []
for category, symbols in stock_symbols.items():
    all_symbols.extend(symbols)
    print(f"📊 {category}: {', '.join(symbols)}")

print(f"\n🎯 總共將分析 {len(all_symbols)} 支股票/標的")

# 設定時間參數
PERIOD = "1d"      # 抓取最近1天的資料
INTERVAL = "1m"    # 1分鐘間隔
MAX_RETRIES = 3    # 最大重試次數

print(f"\n⏰ 時間設定:")
print(f"   時間範圍: {PERIOD}")
print(f"   資料間隔: {INTERVAL}")
print(f"   預計資料點數: ~390點 (6.5小時交易時間 × 60分鐘)")

## 3. 抓取1分鐘間隔的股票資訊 🔄

使用yfinance API抓取股票資料，並加上錯誤處理和進度顯示。我們將：
1. 批量抓取所有股票資料
2. 加入重試機制處理網路問題
3. 顯示抓取進度和統計資訊
4. 儲存完整的股票資訊到字典中

In [ ]:
def fetch_stock_data(symbol, period=PERIOD, interval=INTERVAL, max_retries=MAX_RETRIES):
    """
    抓取單一股票的資料，包含重試機制
    """
    for attempt in range(max_retries):
        try:
            # 創建股票物件
            stock = yf.Ticker(symbol)
            
            # 抓取歷史資料
            hist_data = stock.history(period=period, interval=interval)
            
            if hist_data.empty:
                print(f"⚠️  {symbol}: 無歷史資料")
                return None
            
            # 獲取股票基本資訊
            try:
                info = stock.info
                company_name = info.get('longName', info.get('shortName', symbol))
                sector = info.get('sector', 'Unknown')
                market_cap = info.get('marketCap', 0)
            except:
                company_name = symbol
                sector = 'Unknown'
                market_cap = 0
            
            # 計算基本統計資訊
            latest_price = hist_data['Close'].iloc[-1]
            daily_change = hist_data['Close'].iloc[-1] - hist_data['Open'].iloc[0]
            daily_change_pct = (daily_change / hist_data['Open'].iloc[0]) * 100
            avg_volume = hist_data['Volume'].mean() if 'Volume' in hist_data.columns else 0
            
            # 回傳完整資料
            return {
                'symbol': symbol,
                'company_name': company_name,
                'sector': sector,
                'market_cap': market_cap,
                'data': hist_data,
                'latest_price': latest_price,
                'daily_change': daily_change,
                'daily_change_pct': daily_change_pct,
                'avg_volume': avg_volume,
                'data_points': len(hist_data),
                'fetch_time': datetime.now()
            }
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"❌ {symbol}: 抓取失敗 ({str(e)})")
                return None
            else:
                print(f"🔄 {symbol}: 重試中... (嘗試 {attempt + 1}/{max_retries})")
                continue

print("✅ 股票資料抓取函數已定義完成")

In [ ]:
# 開始批量抓取股票資料
print("🚀 開始抓取股票資料...")
print("=" * 60)

stock_data = {}
successful_count = 0
failed_symbols = []

start_time = datetime.now()

for i, symbol in enumerate(all_symbols, 1):
    print(f"📡 [{i}/{len(all_symbols)}] 正在抓取 {symbol}...", end=" ")
    
    result = fetch_stock_data(symbol)
    
    if result:
        stock_data[symbol] = result
        successful_count += 1
        print(f"✅ 成功 ({result['data_points']} 筆資料)")
    else:
        failed_symbols.append(symbol)
        print(f"❌ 失敗")

end_time = datetime.now()
elapsed_time = (end_time - start_time).total_seconds()

print("=" * 60)
print(f"📊 抓取結果統計:")
print(f"   ✅ 成功: {successful_count} 支股票")
print(f"   ❌ 失敗: {len(failed_symbols)} 支股票")
print(f"   ⏱️  總耗時: {elapsed_time:.1f} 秒")
print(f"   📈 成功率: {(successful_count/len(all_symbols)*100):.1f}%")

if failed_symbols:
    print(f"\n❌ 失敗的股票代碼: {', '.join(failed_symbols)}")

print(f"\n🎉 資料抓取完成！共獲得 {len(stock_data)} 支股票的資料")

## 4. 檢視與分析取得的資料內容 🔍

現在我們來檢查抓取到的資料品質和內容，包括：
1. 資料結構和欄位說明
2. 資料完整性檢查
3. 基本統計分析
4. 資料視覺化展示

In [ ]:
# 檢視第一支股票的資料結構
if stock_data:
    first_symbol = list(stock_data.keys())[0]
    first_data = stock_data[first_symbol]
    
    print(f"📋 資料結構示例 ({first_symbol}):")
    print("=" * 50)
    print(f"🏢 公司名稱: {first_data['company_name']}")
    print(f"🏭 產業類別: {first_data['sector']}")
    print(f"💰 最新價格: ${first_data['latest_price']:.2f}")
    print(f"📊 今日漲跌: {first_data['daily_change_pct']:+.2f}%")
    print(f"📈 資料點數: {first_data['data_points']}")
    
    print(f"\n📊 價格資料欄位:")
    sample_df = first_data['data']
    print(f"   欄位: {list(sample_df.columns)}")
    print(f"   時間範圍: {sample_df.index[0]} 到 {sample_df.index[-1]}")
    print(f"   資料形狀: {sample_df.shape}")
    
    print(f"\n🔍 前5筆資料預覽:")
    display(sample_df.head())
    
    print(f"\n🔍 最後5筆資料預覽:")
    display(sample_df.tail())
else:
    print("❌ 沒有可用的股票資料")

In [ ]:
# 創建所有股票的摘要統計表
summary_data = []

for symbol, data in stock_data.items():
    summary_data.append({
        '股票代碼': symbol,
        '公司名稱': data['company_name'][:20] + '...' if len(data['company_name']) > 20 else data['company_name'],
        '產業': data['sector'],
        '最新價格': f"${data['latest_price']:.2f}",
        '今日漲跌%': f"{data['daily_change_pct']:+.2f}%",
        '平均成交量': f"{data['avg_volume']:,.0f}" if data['avg_volume'] > 0 else 'N/A',
        '資料點數': data['data_points'],
        '抓取時間': data['fetch_time'].strftime('%H:%M:%S')
    })

summary_df = pd.DataFrame(summary_data)

print("📊 所有股票資料摘要:")
print("=" * 80)
display(summary_df)

# 統計分析
print(f"\n📈 統計分析:")
print(f"   📊 總股票數: {len(stock_data)}")
print(f"   📈 平均資料點數: {summary_df['資料點數'].mean():.0f}")
print(f"   📉 最少資料點數: {summary_df['資料點數'].min()}")
print(f"   📈 最多資料點數: {summary_df['資料點數'].max()}")

In [ ]:
# 為每支股票計算技術指標
def calculate_technical_indicators(df):
    """計算常用的技術指標"""
    if df.empty or 'Close' not in df.columns:
        return df
    
    # 移動平均線
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    
    # RSI (相對強弱指標)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # 價格變化百分比
    df['Price_Change_Pct'] = df['Close'].pct_change() * 100
    
    # 布林通道 (如果有足夠資料)
    if len(df) >= 20:
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    
    return df

# 為所有股票添加技術指標
print("🔧 正在計算技術指標...")

for symbol in stock_data:
    try:
        stock_data[symbol]['data'] = calculate_technical_indicators(stock_data[symbol]['data'])
        print(f"✅ {symbol}: 技術指標計算完成")
    except Exception as e:
        print(f"❌ {symbol}: 技術指標計算失敗 - {str(e)}")

print("✅ 所有技術指標計算完成！")

In [ ]:
# 創建綜合視覺化分析
def create_stock_analysis_charts():
    """創建股票分析圖表"""
    if not stock_data:
        print("❌ 沒有可用的資料進行視覺化")
        return
    
    # 選擇前6支有資料的股票進行視覺化
    valid_stocks = [(symbol, data) for symbol, data in stock_data.items() 
                   if not data['data'].empty][:6]
    
    if len(valid_stocks) == 0:
        print("❌ 沒有有效的股票資料")
        return
    
    # 創建子圖
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('📊 股票1分鐘間隔資料分析儀表板', fontsize=16, fontweight='bold')
    
    for i, (symbol, stock_info) in enumerate(valid_stocks):
        row = i // 3
        col = i % 3
        ax = axes[row, col]
        
        df = stock_info['data']
        
        # 繪製價格線和移動平均線
        ax.plot(df.index, df['Close'], label='收盤價', linewidth=1.5, alpha=0.8)
        
        if 'MA5' in df.columns:
            ax.plot(df.index, df['MA5'], label='MA5', linewidth=1, alpha=0.7)
        if 'MA20' in df.columns:
            ax.plot(df.index, df['MA20'], label='MA20', linewidth=1, alpha=0.7)
        
        # 設定標題和格式
        change_pct = stock_info['daily_change_pct']
        color = 'red' if change_pct >= 0 else 'green'
        ax.set_title(f'{symbol} ({change_pct:+.2f}%)', 
                    fontweight='bold', color=color, fontsize=12)
        
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
        
        # 格式化x軸
        ax.tick_params(axis='x', rotation=45, labelsize=8)
        ax.tick_params(axis='y', labelsize=8)
    
    # 調整佈局
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()

# 執行視覺化
create_stock_analysis_charts()

In [ ]:
# 額外分析：找出今日表現最好和最差的股票
print("🏆 今日表現分析:")
print("=" * 50)

if stock_data:
    # 按漲跌幅排序
    performance_data = [(symbol, data['daily_change_pct']) 
                       for symbol, data in stock_data.items()]
    performance_data.sort(key=lambda x: x[1], reverse=True)
    
    print("📈 今日漲幅前5名:")
    for i, (symbol, change_pct) in enumerate(performance_data[:5], 1):
        company_name = stock_data[symbol]['company_name']
        print(f"   {i}. {symbol} ({company_name[:15]}...): {change_pct:+.2f}%")
    
    print("\n📉 今日跌幅前5名:")
    for i, (symbol, change_pct) in enumerate(performance_data[-5:], 1):
        company_name = stock_data[symbol]['company_name']
        print(f"   {i}. {symbol} ({company_name[:15]}...): {change_pct:+.2f}%")

# 資料匯出功能
print(f"\n💾 資料匯出:")
print("=" * 30)

# 匯出摘要資料到CSV
if stock_data:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 匯出摘要
    summary_df.to_csv(f'stock_summary_{timestamp}.csv', 
                     index=False, encoding='utf-8-sig')
    print(f"✅ 摘要資料已匯出: stock_summary_{timestamp}.csv")
    
    # 匯出個別股票的詳細資料
    exported_count = 0
    for symbol, stock_info in stock_data.items():
        try:
            df = stock_info['data']
            if not df.empty:
                filename = f'stock_data_{symbol}_{timestamp}.csv'
                df.to_csv(filename, encoding='utf-8-sig')
                exported_count += 1
        except Exception as e:
            print(f"❌ {symbol} 匯出失敗: {str(e)}")
    
    print(f"✅ 已匯出 {exported_count} 支股票的詳細資料")
    
    # 儲存完整資料為pickle格式（用於後續分析）
    import pickle
    with open(f'complete_stock_data_{timestamp}.pkl', 'wb') as f:
        pickle.dump(stock_data, f)
    print(f"✅ 完整資料已儲存: complete_stock_data_{timestamp}.pkl")

print(f"\n🎯 分析完成！")
print(f"📊 成功分析了 {len(stock_data)} 支股票的1分鐘間隔資料")
print(f"📁 所有結果已儲存到當前目錄")

## 🎯 結論與下一步建議

### 📋 本次分析總結
通過這個notebook，我們成功使用了data science領域最普遍的方法來抓取和分析股票資料：

1. **資料來源**: 使用`yfinance` - 最受歡迎的免費股票API
2. **資料處理**: 使用`pandas`進行資料清理和分析
3. **視覺化**: 使用`matplotlib`和`seaborn`創建專業圖表
4. **技術指標**: 實現了常用的技術分析指標
5. **資料匯出**: 支援多種格式的資料儲存

### 🚀 進階功能建議

如果你想要擴展這個分析，可以考慮以下方向：

#### 📈 技術分析擴展
- 添加更多技術指標（MACD、KD、威廉指標等）
- 實現交易信號識別
- 加入支撐阻力位分析

#### 🤖 機器學習應用
- 使用歷史資料訓練價格預測模型
- 實現異常檢測算法
- 建立投資組合優化系統

#### 📊 即時監控
- 建立即時資料更新機制
- 加入價格預警功能
- 整合新聞情緒分析

#### 🌐 Web應用
- 使用Streamlit或Dash建立互動式網頁
- 實現RESTful API服務
- 加入使用者個人化功能

### 💡 使用建議
- 定期執行此notebook以獲取最新資料
- 根據需求調整股票清單和時間範圍
- 結合基本面分析進行更全面的投資研究

**⚠️ 免責聲明**: 本分析僅供教育和研究用途，不構成投資建議。投資有風險，請謹慎評估。